# Process vibration data
* Features are FFT of segment of vibration data with n_freqs
* Split into train, validation and test sets
* For each speed, make distinct training set with that speed left out but retained for testing
* Spatial/static uses single samples/frames. Labels are the texture (here 13)

To run, first edit dir_data to path where data is stored

Because of the large amount of data, a temporary folder is used for the training data, so it can be deleted after training the model

In [1]:
import os, pickle
import numpy as np
from scipy.io import wavfile

def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def open_obj(name):
    with open(name + '.pkl', 'rb') as f:
        obj = pickle.load(f)
    return obj

def getSpectra(samples, s1, s2, sampleRte):
    spect = np.fft.fft(samples[s1:s2])
    mod = np.sqrt(np.power(spect.real,2)+np.power(spect.imag,2)) 
    mod_norm = 2*(mod/(s2-s1))
    freqs = (np.fft.fftfreq(mod.shape[0]))*sampleRte
   
    return freqs, mod_norm

In [2]:
dir_data = os.environ["DATAPATH"] + r"/open/afferents-tactile-textures-jrsi2022"
dir_temp = os.environ["TEMPPATH"] 

speeds = ['10','20','30','40','50','60','70','80','90','100']
textures = ['0','0.5','1','1.5','2','2.5','3','3.5','4','4.5','5','5.5','6']

n_speeds = len(speeds)
n_textures = len(textures)

t_data = 60 # duration of data
fs = 44100 # sampling rate per second
t_segment = 1 # duration of data segment
n_samples = t_segment*fs # number of samples per data segment
n_data = 119 # number of data segments per speed & texture (assumes stride 0.5)
n_freqs = 4000 # number of FFT samples kept

n_train = n_data*(n_speeds-1)*n_textures
n_val = int(np.floor(n_data/2))*n_textures
n_test = int(np.floor(n_data/2)+1)*n_textures

In [3]:
data = np.zeros((n_speeds, n_textures, t_data*fs))
for i in range(n_speeds):
    for j in range(n_textures):
        _, wav = wavfile.read(dir_data + rf"/{textures[j]}/{speeds[i]}/audio.wav")
        data[i,j,:] = wav[-t_data*fs:,1]

### Split into samples

In [4]:
# split into individual samples and perform FFT
X_audio = np.zeros((n_speeds, n_textures, n_data, n_samples))
X_fft = np.zeros((n_speeds, n_textures, n_data, n_samples))
for i in range(n_speeds):
    print(i, end=' ')
    for j in range(n_textures):
        for n, t in enumerate(np.linspace(0, t_data-t_segment, n_data)):
            X_audio[i,j,n,:] = data[i,j, int(t*fs) : int(t*fs)+n_samples]
            f, X_fft[i,j,n,:] = getSpectra(X_audio[i,j,n,:], 0, len(X_audio[i,j,n,:]), n_samples) # perform FFT

# normalise and truncate the FFT
X_fft = X_fft[..., :n_freqs]
X_fft /= np.amax(X_fft)

0 1 2 3 4 5 6 7 8 9 

### Split into train/test/val

In [7]:
os.makedirs(dir_temp + r"/vibration", exist_ok=True)
os.makedirs(dir_data + r"/processed/vibration", exist_ok=True)

for i in range(n_speeds): 
    dir_train_i = dir_temp + rf"/vibration/{i}"
    dir_test_i = dir_data + rf"/processed/vibration/{i}"
    os.makedirs(dir_train_i, exist_ok=True)
    os.makedirs(dir_test_i, exist_ok=True)

    X_train = np.zeros((n_train, n_freqs))
    X_val = np.zeros((n_val, n_freqs))
    X_test = np.zeros((n_test, n_freqs))  

    y_train_texture = np.zeros((n_train, n_textures))
    y_val_texture = np.zeros((n_val, n_textures))
    y_test_texture = np.zeros((n_test, n_textures))
    
    y_train_speed = np.zeros((n_train, 1))    
    y_val_speed = np.zeros((n_val, 1))
    y_test_speed = np.zeros((n_test, 1))
    
    n = 0
    for j in range(n_speeds):
        if not j==i:
            X_train[n*n_data*n_textures : (n+1)*n_data*n_textures, :] = np.concatenate(X_fft[j,:,:,:])
            for k in range(n_textures):
                y_train_texture[n_data*(n*n_textures+k) : n_data*(n*n_textures+k+1), k] = 1
                y_train_speed[n_data*(n*n_textures+k) : n_data*(n*n_textures+k+1)] = j/10
            n+=1
        else:
            val_ind = np.random.choice(n_data, int(np.floor(n_data/2)), replace=False)
            n_1 = 0
            n_2 = 0
            for k in range(n_data):
                if k in val_ind:
                    X_val[n_1*n_textures:(n_1+1)*n_textures,:] = X_fft[j,:,k,:]
                    for l in range(n_textures):
                        y_val_texture[n_1*n_textures+l,l] = 1
                        y_val_speed[n_1*n_textures+l] = i/n_speeds
                    n_1+=1
                else:
                    X_test[n_2*n_textures:(n_2+1)*n_textures,:] = X_fft[j,:,k,:]
                    for l in range(n_textures):
                        y_test_texture[n_2*n_textures+l,l] = 1
                        y_test_speed[n_2*n_textures+l] = i/n_speeds
                    n_2+=1        
                    
    save_obj(X_train, dir_train_i + r"/X_train")
    save_obj(X_val, dir_train_i + r"/X_val")
    save_obj(X_test, dir_test_i + r"/X_test")
    
    save_obj(y_train_texture, dir_train_i + r"/y_train")
    save_obj(y_val_texture, dir_train_i + r"/y_val")
    save_obj(y_test_texture, dir_test_i + r"/y_test")